In [ ]:
pip install musdb

In [ ]:
import musdb
help(musdb)

In [ ]:
from datetime import datetime
datetime.now()

In [ ]:
b = BandSplit([20,20,20,30,30,30,30,30,30,30,30,30,30,50,50,50,50,70,70,100,100,125],128)


In [ ]:
b.to('cuda:0')

In [ ]:
y[0].shape

In [ ]:
b.double()
c = b(y[0])
c.shape



In [ ]:
c = c.permute(0,1,3,2)

In [ ]:
conv1 = ConvolutionLayer(22, 22, (1,7), (1,3),).to('cuda:0')

In [ ]:
conv2 = ConvolutionLayer(22, 22, (3,3), (2,2)).to('cuda:0')

In [ ]:
chromas = y[1]
mfccs = y[2]
batch_size = 32

chromas = chromas.reshape(batch_size, 1, 12, 431)
chromas.shape

In [ ]:
conv3 = ConvolutionLayer(1,8,kernel_size=(1,11),stride=(1,6)).to('cuda:0')
conv4 = ConvolutionLayer(8,22,kernel_size=(1,3),stride=(1,2)).to('cuda:0')

In [ ]:
conv3(chromas.float()).shape
h = conv3(chromas.float())
j = conv4(h)
j.shape

In [ ]:
d = conv1(c.float())
print(d.shape)
e = conv2(d)
print(e.shape)
f = conv2(e)
f.shape

In [ ]:
model.to('cuda:0');

In [ ]:
model = Model([20,20,20,30,30,30,30,30,30,30,30,30,30,50,50,50,50,70,70,100,100,125],128)

In [ ]:
y[0].double();

In [ ]:
y[1].shape

In [ ]:
out, chromas, mfccs = model(y[0].double(), y[1], y[2])

In [ ]:
mfccs.shape

In [ ]:
out.shape

In [ ]:
z = torch.cat((mfccs,out,torch.zeros(32,22,1,70).to('cuda:0')), 2)
z.shape

In [ ]:
bblstm = BandBiLSTM(96, 22, 48).to('cuda:0')
bblstm.double()

In [ ]:
z.reshape(32,22,70*96)

In [ ]:
blstmt1 = bblstm(z)

In [ ]:
blstmt1.shape

In [ ]:
tblstm = TemporalBiLSTM(70, 96, 35).to('cuda:0')

In [ ]:
tblstm.double()

In [ ]:
blstmt2 = tblstm(blstmt1)

In [ ]:
blstmt2.shape

In [ ]:
norm = nn.GroupNorm(22, 22).to('cuda:0').double()

In [ ]:
norm(z).mean()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
ablstms = AlternatingBLSTMs(22, 70, 96).to('cuda:0').double()

In [ ]:
z = z.to('cuda:1')
ablstms(z)

In [ ]:
bblstm(z).permute(0,2,3,1).shape

In [ ]:
bblstm

In [ ]:
chromas.shape

In [ ]:
deconv = TransposeConvolutionLayer(22, 22, (2,2), (2,2)).to('cuda:0')

In [ ]:
deconv.double()
deconv(out).shape

In [ ]:
total_params = sum(
	param.numel() for param in conv4.parameters()
)
total_params

In [ ]:
d = blstm1(c.float())

In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F

def test_module(module, input_size):
    """
    Test an nn.Module subclass with random input data of the given size.
    
    :param module: The nn.Module subclass to test.
    :param input_size: A tuple of integers specifying the size of the input data.
    """
    # Create random input data with the specified size
    input_data = torch.randn(input_size)
    print(f'Input shape: {input_data.shape}')

    # Run the input data through the module
    output = module(input_data)
    print(f'Output shape: {output.shape}')
    
    
    
class Model(torch.nn.Module):
    def __init__(self, bandwidths, N, kernel1 = (1,7), stride1=(1,3), kernel2 = (3,3), stride2 = (2,2), device = 'cuda:0', dtype = 'double', n_mels = 32):
        super(Model, self).__init__()
        self.device = device
        self.N = N
        self.n_mels = n_mels
        self.K = len(bandwidths)
        self.bandsplit = BandSplit(bandwidths, N).to(device)
        self.bandsplit.double()
        self.conv1 = ConvolutionLayer(self.K, self.K, kernel1, stride1).to(device)
        self.conv1.double()
        self.conv2 = ConvolutionLayer(self.K, self.K, kernel2, stride2).to(device)
        self.conv2.double()
        self.conv3 = ConvolutionLayer(1,8,kernel_size=(1,11),stride=(1,3)).to(device)
        self.conv3.double()
        self.conv4 = ConvolutionLayer(8,22,kernel_size=(1,3),stride=(1,2)).to(device)
        self.conv4.double()
        self.conv5 = ConvolutionLayer(1,8,kernel_size=(1,11),stride=(1,6)).to(device)
        self.conv5.double()
        self.conv6 = ConvolutionLayer(8,22,kernel_size=(1,3),stride=(1,2)).to(device)
        self.conv6.double()
        self.bandblstm1 = BandBiLSTM()
        
    def forward(self, X, chromas, mfccs):
        X = self.bandsplit(X)
        #Shape: torch.Size([32, 22, 431, 128]) (batch_size, num_bands, time_steps, freq_N)
        X = self.conv1(X)
        X = self.conv2(X)
        batch_size = X.shape[0]
        mfccs = mfccs.reshape(batch_size,1,self.n_mels,-1)
        mfccs = mfccs.to(self.device)
        mfccs = self.conv3(mfccs)
        mfccs = self.conv4(mfccs)
        chromas = chromas.reshape(batch_size,1,12,-1)
        chromas = chromas.to(self.device)
        chromas = self.conv5(chromas)
        chromas = self.conv6(chromas)
        xmfccs = torch.cat(X, mfccs, 2)
        
        return X, chromas, mfccs
        
        
class ConvolutionLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, dtype='double'):
        super(ConvolutionLayer, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        x = self.conv(x)
        return x
    
class TransposeConvolutionLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dtype='double'):
        super(TransposeConvolutionLayer, self).__init__()
        self.conv = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        x = self.conv(x)
        return x
    
    
class AlternatingBLSTMs(nn.Module):
    def __init__(self, num_bands, time_steps, N, axis=1):
        super(AlternatingBLSTMs, self).__init__()
        self.band_blstm = BandBiLSTM(num_bands, time_steps, N, num_bands // 2)
        self.temporal_blstm = TemporalBiLSTM(num_bands, time_steps, N, time_steps // 2)
        self.num_bands = num_bands
        self.time_steps = time_steps
        self.N = N
        # hidden size = freq_steps_per_band * time_steps 

    def forward(self, x):
        # Input shape: (batch_size, num_bands, N, time_steps)
        # Prepare for Band BLSTM: shape = (batch_size, num_bands, N * time_steps)
        batch_size = x.shape[0]
        x = x.reshape(batch_size, self.num_bands, -1)
        x = self.band_blstm(x)
        x = x.reshape(batch_size, self.N, -1)
        x = self.temporal_blstm(x)
        
        #x += residual
        # Return the output of the module
        return x    
# This class defines a module that runs the input, with shape (num_bands, num_timesteps, N), through a normalization layer, then a temporal biLSTM, then a fully connected layer.
# Then, the output of that layer is of the same shape as the input to the module, which will be fed into a similar structure, but this time with a band biLSTM, following the same normalization, biLSTM, FC structure.
class BandBiLSTM(nn.Module):
    def __init__(self, num_bands, time_steps, N, hidden_size, axis=1):
        super(BandBiLSTM, self).__init__()
        self.norm = nn.GroupNorm(num_bands, num_bands)
        self.input_size = time_steps * N
        self.hidden_size = self.input_size // 2
        self.bilstm = nn.LSTM(self.input_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(N, N)
        self.axis = axis
        self.N = N
        self.num_bands = num_bands
        self.hidden_size = hidden_size
        self.time_steps = time_steps
        # hidden size = freq_steps_per_band * time_steps 

    def forward(self, x):
        print("BAND INPUT SHAPE: ", x.shape)
        # (batch_size,time_steps, num_bands, N)
        x = self.norm(x)
        residual = x.clone().detach()
        x, lstm_vars = self.bilstm(x)
        # (batch_size, num_bands, 2 * hidden_size)
        x = x.reshape(batch_size, self.num_bands, self.time_steps, self.N)
        # (batch_size, num_bands, time_steps, N)
        x = self.fc(x)
        # (batch_size, num_bands, time_steps, N)
        print(x.shape)
        #x += residual
        # Return the output of the module
        return x
    
class TemporalBiLSTM(nn.Module):
    def __init__(self, num_bands, time_steps, N, hidden_size, axis=1):
        super(TemporalBiLSTM, self).__init__()
        self.norm = nn.GroupNorm(time_steps, time_steps)
        self.input_size = num_bands * N
        self.hidden_size = self.input_size // 2
        self.bilstm = nn.LSTM(self.input_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(N, N)
        self.axis = axis
        self.N = N
        self.time_steps = time_steps
        self.hidden_size = hidden_size
        self.num_bands = num_bands
        # hidden size = freq_steps_per_band * time_steps 

    def forward(self, x):
        print("TEMPORAL INPUT SHAPE: ", x.shape)
        # (batch_size,time_steps, num_bands, N)
        x = self.norm(x)
        residual = x.clone().detach()
        x, lstm_vars = self.bilstm(x)
        # (batch_size, num_bands, 2 * hidden_size)
        x = x.reshape(batch_size, self.num_bands, self.time_steps, self.N)
        # (batch_size, num_bands, time_steps, N)
        x = self.fc(x)
        # (batch_size, num_bands, time_steps, N)
        print(x.shape)
        #x += residual
        # Return the output of the module
        return x, lstm_vars

class BandSplit(torch.nn.Module):
    # Input shape: torch.Size([16, 2, 1025, 431, 2])
    def __init__(self, bandwidths, N):
        # bandwidth
        super(BandSplit, self).__init__()
        self.bandwidths = bandwidths
        self.norm_layers = torch.nn.ModuleList([torch.nn.LayerNorm(2 * bandwidth) for bandwidth in self.bandwidths])
        self.fc_layers = torch.nn.ModuleList([torch.nn.Linear(2 * bandwidth, N) for bandwidth in self.bandwidths])

    def forward(self, X):
        subband_spectrograms = []
        K = len(self.bandwidths)
        for i in range(K):
            start_index = sum(self.bandwidths[:i])
            end_index = start_index + self.bandwidths[i]
            subband_spectrogram = X[:, :,start_index:end_index, :]
            subband_spectrogram = subband_spectrogram.permute(0,1,4,2,3)
            subband_spectrogram = subband_spectrogram.reshape(2 * X.shape[0], X.shape[3], 2 * self.bandwidths[i])
            subband_spectrograms.append(subband_spectrogram)

        subband_features = []
        for i in range(K):
            norm_output = self.norm_layers[i](subband_spectrograms[i])
            fc_output = self.fc_layers[i](norm_output)
            subband_features.append(fc_output)

        Z = torch.stack(subband_features, dim=1)
        Z = Z.permute(0,1,3,2)
        return Z

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

def test_module(module, input_size):
    """
    Test an nn.Module subclass with random input data of the given size.
    
    :param module: The nn.Module subclass to test.
    :param input_size: A tuple of integers specifying the size of the input data.
    """
    # Create random input data with the specified size
    input_data = torch.randn(input_size)
    print(f'Input shape: {input_data.shape}')

    # Run the input data through the module
    output = module(input_data)
    print(f'Output shape: {output.shape}')



class BandSplit(torch.nn.Module):
    def __init__(self, bandwidths, N):
        # bandwidth
        super(my_model, self).__init__()
        self.bandwidths = bandwidths
        self.norm_layers = torch.nn.ModuleList([torch.nn.LayerNorm(2 * bandwidth) for bandwidth in self.bandwidths])
        self.fc_layers = torch.nn.ModuleList([torch.nn.Linear(2 * bandwidth, N) for bandwidth in self.bandwidths])

    def forward(self, X):
        subband_spectrograms = []
        K = len(self.bandwidths)
        for i in range(K):
            start_index = sum(self.bandwidths[:i])
            end_index = start_index + self.bandwidths[i]
            subband_spectrogram = X[:, start_index:end_index, :]
            subband_spectrograms.append(subband_spectrogram)

        subband_features = []
        for i in range(K):
            norm_output = self.norm_layers[i](subband_spectrograms[i])
            fc_output = self.fc_layers[i](norm_output)
            subband_features.append(fc_output)

        Z = torch.stack(subband_features, dim=1)

        return Z


# This class defines a module that runs the input, with shape (num_bands, num_timesteps, N), through a normalization layer, then a temporal biLSTM, then a fully connected layer.
# Then, the output of that layer is of the same shape as the input to the module, which will be fed into a similar structure, but this time with a band biLSTM, following the same normalization, biLSTM, FC structure.
class GeneralBiLSTMUnit(nn.Module):
    def __init__(self, N, hidden_size, axis=1):
        super(GeneralBiLSTMUnit, self).__init__()
        self.norm = nn.GroupNorm(1, N)
        self.bilstm = nn.LSTM(N, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, N)
        self.axis = axis

    def forward(self, x):
        # Normalize the input
        x = self.norm(x)

        # Apply the temporal biLSTM layer
        x, _ = self.bilstm(x.transpose(0,1))

        # Apply the fully connected layer
        x = self.fc(x)
        x = x.transpose(0,1)
        
        x += input 
        # Return the output of the module
        return x

class generalTransformerUnit(nn.Module):
    def __init__(self, N, hidden_size, axis=1):
        super(GeneralBiLSTMUnit, self).__init__()
        self.norm = nn.GroupNorm(1, N)
        self.bilstm = nn.(N, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, N)
        self.axis = axis

    def forward(self, x):
        # Normalize the input
        x = self.norm(x)

        # Apply the temporal biLSTM layer
        x, _ = self.transformer(x.transpose(0,1))

        # Apply the fully connected layer
        x = self.fc(x)
        x = x.transpose(0,1)
        
        x += input 
        # Return the output of the module
        return x


class InterleavedBiLSTMs(nn.Module):
    def __init__(self, N, hidden_size):
        super(InterleavedBiLSTMs, self).__init__()
        self.temporal_bilstm = GeneralBiLSTMUnit(N, hidden_size)
        self.band_bilstm = GeneralBiLSTMUnit(N, hidden_size)

    def forward(self, x):
        x = self.temporal_bilstm(x)
        x = self.band_bilstm(x)
        return x



In [ ]:
class my_model(nn.Module):
    def __init__(self, x):
        super().__init__()
        self.linear = nn.LayerNorm(x.shape)
    def forward(self, x):
        x = self.linear(x)
        return x

In [ ]:
test_module()

In [2]:
def collate(batch):
    # we define a custom function that overrides the PyTorch built in collate in order to make sure
    # our input to the model matches the dimensionality we want.
    # our input to the function is a list of (input, label) tuples. The list will have size = batch_size as defined in the DataLoader
    # We will use a batch_size of 1 to receive a list of tuples (in this case a list with 1 tuple), each tuple has two elements:
    # First, the input, a tensor with shape (batch_size, stft_dim_F, stft_dim_T, 2)
    # Second, the label, a tensor with shape (num_stems - 1, batch_size, stft_dim_F, stft_dim_T, 2)
    input, chromas, mfccs, labels = batch[0]
    return input, chromas, mfccs, labels

In [19]:
T = Transforms()

In [20]:
musValidation = MusdbDataset(T, "../../musdb18/", "valid", "train")

KeyboardInterrupt: 

In [25]:
mus = musdb.DB("../../musdb18/", subsets = "train", split = "valid")

In [30]:
trainloader = torch.utils.data.DataLoader(mus, batch_size=1, collate_fn=collate)

In [35]:
min([int(track.duration * 44100) for track in mus.tracks])

7204704

In [37]:
musTraining = MusdbDataset(T, "../../musdb18/")

KeyboardInterrupt: 

In [ ]:
pip install musdb

In [ ]:

trainloader = torch.utils.data.DataLoader(musValidation, batch_size=1, collate_fn=collate)

In [31]:

y = 0
for x in trainloader:
    y = x
    break


TypeError: cannot unpack non-iterable MultiTrack object

In [ ]:
y[1].shape

In [ ]:
y[0].shape

In [ ]:
y[0].shape

In [ ]:
model = my_model(torch.randn([]))
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/train_{}'.format(timestamp))
train = Train(musTraining, musValidation, writer, optimizer, model, 0.001, SSloss, )

In [36]:
"""
This is the file to retreive the data from MusDB, it allows us to choose between the HQ dataset with .wav files
or the compressed dataset with .stem.mp4 files. It also allows us to choose between the train and test subsets of the dataset.
The class MusdbDataset is a subclass of torch.utils.data.Dataset, which allows us to use the PyTorch DataLoader class
to load the data in batches. The __init__ method initializes the dataset, and the __getitem__ method returns the data
for a given index. The __len__ method returns the length of the dataset.

NOTES TO SELF: Fix threshold to set only off mix source
                OTHER functions than RMS
                Make self.batch_size
"""

import musdb
import torch
import torch.nn as nn
# Padding, constant batch size


class MusdbDataset(torch.utils.data.Dataset):
    def __init__(self, transforms, musdb_root, split='train', subset='train', is_wav=False, sample_rate=44100, segment_length = 10, segment_chunks = 10, discard_low_energy = True, segment_overlap = 0.5, drop_percentile =  0.1, chunks_below_percentile = 0.5):
        # Check if a GPU is available
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        assert(subset == 'train' or subset == 'test')
        self.mode = subset
        self.mus = musdb.DB(musdb_root, subsets=subset, split=split, is_wav=is_wav)
        self.durations = dict()
        self.init_durations()
        self.transforms = transforms
        self.sample_rate = sample_rate
        self.shortest_duration = self.shortest_duration_in_samples(self.mus)
        self.split = split
        self.discard_low_energy = discard_low_energy
        self.segment_length = segment_length
        self.segment_chunks = segment_chunks
        self.chunks_below_percentile = chunks_below_percentile
        self.segment_overlap = segment_overlap
        self.drop_percentile = drop_percentile
        self.segment_samples = int(self.segment_length * self.sample_rate)
        self.stft_size = self.STFT_dimensions(self.segment_samples)
        self.chunk_samples = int(self.segment_samples / self.segment_chunks)
        self.num_stems = self.mus.tracks[0].stems.shape[0]
        self.num_channels = self.mus.tracks[0].stems.shape[2]
        self.batch_size = self.find_batch_size()


    def __len__(self):
        return len(self.mus.tracks)


    def __getitem__(self, idx):
        # this function should return a batch of segment STFTs from the song as well as their stem STFTs.
        track = self.mus.tracks[idx]
        # stems is a list of the stems of the track, in the order of the stems in the track
        stems = torch.as_tensor(track.stems, device = self.device)
        print("STEMS SHAPE :", stems.shape)
        if self.mode == 'train':
            segments = self._batchize_training_item(stems)
        else:
            segments = self._batchize_testing_item(stems)
        print("SEGMENT SHAPE: ", segments.shape)
        # we apply a STFT to each of those segments, and that is how we achieve our constant batch and input sizes.
        stfts = self.stft_segments(segments)
        print("STFTs SHAPE: ", stfts.shape)
        chromas, mfccs = self.chroma_and_mfcc_segments(segments)
        print("Chromas SHAPE: ", chromas.shape)
        # we then return the segments as a torch tensor
        data = stfts[:,0]
        labels = stfts[:,1:]
        mfccs
        print("DATA :", data.shape, " LABELS: ", labels.shape)
        return data, chromas, mfccs, labels

    def _batchize_training_item(self,stems):
        # we need to trim the stems to the shortest duration track, starting from a random location
        stems = self.trim_stems(stems, self.random_start(stems.shape[1]))
        # we split the stems from shape (num_stems, num_samples, num_channels) into a tensor with shape (num_stems, num_segments, num_samples_per_segment, num_channels)
        segments = self.split_track(stems)
        # now we need to drop out the low energy segments
        print(type(segments))
        segments = self.high_energy_segments(segments)
        # then we choose a random and continuous yet constant number of segments from the track minus the dropped segments
        return segments

    def _batchize_testing_item(self,stems):
        # first we make sure the length of the song will produce a whole number of segments by padding with zeros to the end of the next segment.
        stems = self.add_zero_padding(stems)
        # now we extend the song with zeros to make sure we have an equal batch size for every output.
        to_pad = self.longest_duration_in_samples() - stems.shape[1]
        stems = self.add_N_zeros(stems, to_pad)
        # now we split the stems into equal size segments
        segments = self.split_track(stems)
        # we are now ready to operate on the song since we don't want to drop or modify our data as we conserve it to reconstruct our signal.
        return segments

    def collate(batch):
        # we define a custom function that overrides the PyTorch built in collate in order to make sure
        # our input to the model matches the dimensionality we want.
        # our input to the function is a list of (input, label) tuples. The list will have size = batch_size as defined in the DataLoader
        # We will use a batch_size of 1 to receive a list of tuples (in this case a list with 1 tuple), each tuple has two elements:
        # First, the input, a tensor with shape (batch_size, stft_dim_F, stft_dim_T, 2)
        # Second, the label, a tensor with shape (num_stems - 1, batch_size, stft_dim_F, stft_dim_T, 2)
        input, chromas, mfccs, labels = batch[0]
        return input, chromas, mfccs, labels

    def init_durations(self):
        for idx, track in enumerate(self.mus.tracks):
            self.durations[idx] = track.stems.shape[1]
            print(track)
            print(track.duration * 44100)
            print(self.duration[idx])

    def stft_segments(self, segments):
        # this function should take in a list of segments and apply a STFT to each of them
        # it should return a tensor of STFTs, of shape (num_stems, batch_size, STFT_F, STFT_T, num_channels)
        # where batch_size is the number of STFTs we can fit into a batch
        # STFT_F is the number of frequency bins in the STFT
        # STFT_T is the number of time bins in the STFT
        # num_stems is the number of stems in the track
        # num_channels is the number of channels in the track
        # the STFTs should be applied to each segment, and then the segments should be concatenated along the batch_size axis
        stem_stfts = []
        batch_size = self.batch_size
        # we then keep only the first batch_size segments, and apply a STFT to each of them. 
        all_segments = segments[:batch_size, :, :, :].view(self.num_stems, self.num_channels, batch_size, self.segment_samples)
        for i in range(self.num_stems):
            channel_stfts = []
            # we take the first batch of stfts, we reshape it to allow for stft operation using torch.stft and taking advantage of 2D batched STFTs
            for j in range(self.num_channels): 
                # we apply a STFT to each segment
                stem_batch_channel_segments = all_segments[i,j,:,:]
                batch_stfts = self.transforms.stft(stem_batch_channel_segments)
                channel_stfts.append(batch_stfts)
                # we then add the STFT to the list of STFTs
            stem_stfts.append(torch.stack(channel_stfts))
        stem_stfts = torch.stack(stem_stfts)
        return stem_stfts.view(batch_size, self.num_stems, self.num_channels, self.stft_size[0], self.stft_size[1], 2)
        
    def chroma_and_mfcc_segments(self, segments):
        # this function should take in a list of segments and apply a STFT to each of them
        # it should return a tensor of STFTs, of shape (num_stems, batch_size, STFT_F, STFT_T, num_channels)
        # where batch_size is the number of STFTs we can fit into a batch
        # STFT_F is the number of frequency bins in the STFT
        # STFT_T is the number of time bins in the STFT
        # num_stems is the number of stems in the track
        # num_channels is the number of channels in the track
        # the STFTs should be applied to each segment, and then the segments should be concatenated along the batch_size axis
        batch_size = self.batch_size
        # we then keep only the first batch_size segments, and apply a STFT to each of them. 
        all_segments = segments[:batch_size, :, :, :].view(self.num_stems, self.num_channels, batch_size, self.segment_samples)
        channel_chromas = []
        channel_mfccs = []
        # we take the first batch of stfts, we reshape it to allow for stft operation using torch.stft and taking advantage of 2D batched STFTs
        for j in range(self.num_channels): 
            print("123    : ", j)
            # we apply a STFT to each segment
            stem_batch_channel_segments = all_segments[0,j,:,:]
            batch_chromas = self.transforms.chromagram(stem_batch_channel_segments)
            print("1234   :", j)
            batch_mfccs = self.transforms.mel_spectrogram(stem_batch_channel_segments)
            print("12345  :", j)
            channel_chromas.append(batch_chromas)
            channel_mfccs.append(batch_mfccs)
            # we then add the STFT to the list of STFTs
        channel_mfccs = torch.stack(channel_mfccs)
        channel_chromas = torch.stack(channel_chromas)
        return channel_chromas.view(batch_size * self.num_channels, 12, self.stft_size[1]),  channel_mfccs.view(batch_size * self.num_channels, self.transforms.n_mels , self.stft_size[1])
                
        
        
        
        
        
    def trim_stems(self, stems, start):
        print(self.shortest_duration)
        # this function should trim the track to the shortest duration of the track from the start index, it allows looping back across the song.
        if start + self.shortest_duration > stems.shape[1]:
            first_half = stems[:, start:, :]
            remaining = self.shortest_duration - (stems.shape[1] - start)
            second_half = stems[:, :remaining, :]
            return torch.cat((first_half, second_half), axis=1)
        else:
            return stems[:, start:start+self.shortest_duration, :]

    def pad_stems(self, stems):
        length_in_samples = stems.shape[1]
        to_pad = self.longest_duration_in_samples() - length_in_samples

    def find_batch_size(self):
        # this function should tell us how many STFTs we can fit into a batch based on finding the floor power of 2 of the number of STFTs we fit over the duration of the song
        # each STFT will represent a STFT over a fixed segment length.
        num_segments = self.num_segments_in_track(self.shortest_duration)
        # we anticipate a drop of up to twice the drop percentile (impossible, just to be safe) of the segments.
        num_segments = torch.as_tensor(int(num_segments * (1 - 2 * self.drop_percentile)), device = self.device)
        # We return the closest power of two to that anticipated number of segments. Of course we use a floor because we want to fill every batch.
        return 2 ** int(torch.floor(torch.log2(num_segments)))

    def num_segments_in_track(self, duration_in_samples):
        # this function should return the number of segments in the track and consider the overlap factor, self.segment_overlap
        return int(torch.ceil(torch.as_tensor(duration_in_samples / (self.segment_samples * (1 - self.segment_overlap)), device=self.device)))

    def random_start(self, duration_in_samples):
        # this function should return a random start index for the track
        return torch.randint(0, duration_in_samples, (1,))


    def STFT_dimensions(self, duration_in_samples):
        # this function should return the dimensions of the STFT of the shortest track in the dataset
        F = int(self.transforms.n_fft / 2 + 1)
        T = int(torch.ceil(torch.as_tensor(duration_in_samples / self.transforms.hop_length, device = self.device)))
        return (F,T)
        

    def shortest_duration_in_samples(self, mus):
        # this function should return the shortest duration of the stems in the track
        return int(min(self.durations.values()))

    def longest_duration_in_samples(self, mus):
        # this function should return the longest duration of the stems in the track
        return max(min([track.stems.shape[1] for track in mus.tracks]))

    def is_high_energy_segment(self, segment, threshold):
        # this function decides based on the provided threshold whether a sufficient number of chunks in the segment have an energy above the threshold
        mix_chunk_energies = self.segment_chunk_energies(segment)[:]
        return len(torch.argwhere(mix_chunk_energies > threshold)) > (self.chunks_below_percentile * self.segment_chunks)

    def high_energy_segments(self, segments):
        # this function should take in a full track's stems, it will then split the track into segments. 
        # The segments should have an overlap factor of self.segment_overlap.
        # Then, it will split each segment into chunks, and it will calculate the energy of each segment, and store the energy of each segment in a list.
        # With this list, it will calculate the percentile of the energy of the chunks, and it will discard the segments where 25% of the chunks have an energy below the percentile.
        # It will then return the list of segments that have a high enough energy.
        high_energy_indices = []
        threshold = self.segment_energy_threshold(segments)
        for idx, segment in enumerate(segments):
            if self.is_high_energy_segment(segment, threshold):
                high_energy_indices.append(idx)
        high_energy_indices = torch.as_tensor(high_energy_indices, device = self.device)
        return torch.index_select(segments, 0, high_energy_indices)

    def segment_energy_threshold(self, segments):
        # this function should split every segment into self.segment_chunks chunks, and it will calculate the energy of each chunk using the RMS energy function.
        # It will save the energy of each chunk in a list, and it will return the value self.percentile_dropped percentile of the list.
        chunk_energies = []
        for segment in segments:
            chunk_energies.extend(self.segment_chunk_energies(segment))
        chunk_energies = torch.stack(chunk_energies)
        percentile = torch.quantile(chunk_energies, self.drop_percentile, interpolation='midpoint')
        return percentile
            
    def segment_chunk_energies(self, segment):
        # this function should split the segment into self.segment_chunks chunks, and it will calculate the energy of each chunk using the RMS energy function.
        # It will save the energy of each chunk in a list, and it will return the list.
        chunk_energies = []
        segment_samples = segment.shape[1]
        chunk_samples = int(segment_samples / self.segment_chunks)
        for i in range(0, segment_samples, chunk_samples):
            chunk = segment[:, i:i+chunk_samples]
            mix_track = chunk[0,:,:]
            rms = self.transforms.RMS(mix_track)
            chunk_energies.append(rms)
        return torch.stack(chunk_energies).view(self.segment_chunks)

    def split_track(self, stems):
        # this function should take in a full track, and it will split the track into segments. 
        # The segments should have an overlap factor of self.segment_overlap.
        # We add zero padding to the track to make sure that the track is divisible by the segment length.
        # Then, it will split each segment into chunks, and it will return the list of chunks.
        # The input is a tensor with shape (num_stems, num_samples, num_channels)
        # The output is a tensor array with shape (num_stems, num_segments, num_samples_per_segment, num_channels)
        stems = self.add_zero_padding(stems)
        segments = []
        num_samples = stems.shape[1]
        step_in_samples = int(self.segment_samples * (1 - self.segment_overlap))
        for i in range(0, num_samples - step_in_samples, step_in_samples):
            segment = stems[:, i:i+self.segment_samples]
            segments.append(segment)
        segments = torch.stack(segments)
        return segments

    def add_zero_padding(self, stems):
        # this function should add zero padding to the track to make sure that the track is divisible by the segment length and the residue from the overlap.
        # the length of the array has to be segment_length + k * samples_in_steps for some nonnegative integer k.
        num_samples = stems.shape[1]
        step_in_samples = int(self.segment_samples * (1 - self.segment_overlap))
        samples_in_last_segment = num_samples % step_in_samples
        if samples_in_last_segment != 0:
            padding = torch.zeros((stems.shape[0], step_in_samples - samples_in_last_segment, stems.shape[2]), device = self.device)
            return torch.cat((stems, padding), axis=1)
        else:
            return stems

    def add_N_zeros(self, stems, N):
        zeros = torch.zeros((stems.shape[0], N, stems.shape[2]))
        return torch.cat((stems, zeros), axis = 1)

In [14]:
#This file will hold the loss functions for each loss I will test, including a composite loss function

import torch
import torch.nn as nn


def SSloss(predicted_stfts, real_stfts):
    # This function will calculate the loss for a given batch of predicted and real stems
    # The loss will be the sum of the losses for each stem
    # the dimensions of the predicted stems and real stems will be (batch_size, num_stems, stft_dim_F, stft_dim_T,num_channels)
    # The loss will be a scalar
    stft_loss = stft_mean_absolute_error(predicted_stfts, real_stfts)
    # We want to find the signal representation from the STFT we computed. We can do this by taking the inverse STFT
    # of each segment (input in the batch) separately and compare them. We don't need perfect reconstruction for our loss function,
    # but we will need it to evaluate the model.
    predicted_time_signals = batch_time_signals(predicted_stfts)
    real_time_signals = batch_time_signals(real_stfts)
    signal_loss = signal_mean_absolute_error(predicted_time_signals, real_time_signals)
    return signal_loss + stft_loss

def batch_time_signals(stfts):
    # This function should compute the time signals for each segment in the batch
    # input shape (batch_size, num_stems, stft_dim_F, stft_dim_T,num_channels)
    # output shape (batch_size, num_stems, segment_samples, num_channels)
    istft_list = []
    for i in range(stfts.shape[0]):
        for j in range(stfts.shape[1]):
            istft = torch.istft(stfts[i, j], n_fft=2048, hop_length=1024, win_length=2048, window=torch.hann_window(2048, device = torch.device("cuda")))
            istft_list.append(istft)
    istfts = torch.stack(istft_list, dim=0)
    return istfts

    
    
def signal_mean_absolute_error(predicted_time_signal, real_time_signal):
    # This function will calculate the mean absolute error between the predicted and real time signals
    # The dimensions of the predicted and real time signals will be (batch_size, num_stems, stft_dim_F, stft_dim_T,num_channels)
    return torch.mean(torch.abs(predicted_time_signal - real_time_signal))

def stft_mean_absolute_error(predicted_stfts, real_stfts):
    # This function will calculate the mean absolute error between the predicted and real STFTs
    # The dimensions of the predicted and real STFTs will be (batch_size, num_stems, stft_dim_F, stft_dim_T,num_channels)
    return torch.mean(torch.abs(predicted_stfts - real_stfts))



In [15]:
import torch.nn as nn
import torch
import torch.nn.functional as F

def test_module(module, input_size):
    """
    Test an nn.Module subclass with random input data of the given size.
    
    :param module: The nn.Module subclass to test.
    :param input_size: A tuple of integers specifying the size of the input data.
    """
    # Create random input data with the specified size
    input_data = torch.randn(input_size)
    print(f'Input shape: {input_data.shape}')

    # Run the input data through the module
    output = module(input_data)
    print(f'Output shape: {output.shape}')


class SingularValueRegularization(nn.Module):
    def __init__(self, num_singular_values, eps=1e-3):
        super().__init__()
        self.num_singular_values = num_singular_values # number of singular values to keep
        self.eps = eps 
    
    def forward(self, x):
        u, s, v = torch.svd(x) # perform singular value decomposition on x
        s[self.num_singular_values:] = self.eps # set the remaining singular values to epsilon
        x_hat = u @ torch.diag(s) @ v # construct x_hat using the first num_singular_values singular values
        return x_hat


class my_model(torch.nn.Module):
    def __init__(self, bandwidths, N):
        # bandwidth
        super(my_model, self).__init__()
        self.bandwidths = bandwidths
        self.norm_layers = torch.nn.ModuleList([torch.nn.LayerNorm(2 * bandwidth) for bandwidth in self.bandwidths])
        self.fc_layers = torch.nn.ModuleList([torch.nn.Linear(2 * bandwidth, N) for bandwidth in self.bandwidths])

    def forward(self, X):
        subband_spectrograms = []
        K = len(self.bandwidths)
        for i in range(K):
            start_index = sum(self.bandwidths[:i])
            end_index = start_index + self.bandwidths[i]
            subband_spectrogram = X[:, start_index:end_index, :]
            subband_spectrograms.append(subband_spectrogram)

        subband_features = []
        for i in range(K):
            norm_output = self.norm_layers[i](subband_spectrograms[i])
            fc_output = self.fc_layers[i](norm_output)
            subband_features.append(fc_output)

        Z = torch.stack(subband_features, dim=1)

        return Z


# This class defines a module that runs the input, with shape (num_bands, num_timesteps, N), through a normalization layer, then a temporal biLSTM, then a fully connected layer.
# Then, the output of that layer is of the same shape as the input to the module, which will be fed into a similar structure, but this time with a band biLSTM, following the same normalization, biLSTM, FC structure.
class GeneralBiLSTMUnit(nn.Module):
    def __init__(self, N, hidden_size, axis=1):
        super(GeneralBiLSTMUnit, self).__init__()
        self.norm = nn.GroupNorm(1, N)
        self.bilstm = nn.LSTM(N, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, N)
        self.axis = axis

    def forward(self, x):
        # Normalize the input
        x = self.norm(x)

        # Apply the temporal biLSTM layer
        x, _ = self.bilstm(x.transpose(0,1))

        # Apply the fully connected layer
        x = self.fc(x)
        x = x.transpose(0,1)
        
        x += input 
        # Return the output of the module
        return x

class InterleavedBiLSTMs(nn.Module):
    def __init__(self, N, hidden_size):
        super(InterleavedBiLSTMs, self).__init__()
        self.temporal_bilstm = GeneralBiLSTMUnit(N, hidden_size)
        self.band_bilstm = GeneralBiLSTMUnit(N, hidden_size)

    def forward(self, x):
        x = self.temporal_bilstm(x)
        x = self.band_bilstm(x)
        return x




In [6]:
pip uninstall torchaudio

Found existing installation: torchaudio 0.13.0
Uninstalling torchaudio-0.13.0:
  Would remove:
    /opt/conda/lib/python3.10/site-packages/torchaudio-0.13.0.dist-info/*
    /opt/conda/lib/python3.10/site-packages/torchaudio/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import scipy
import torch
import librosa
# import torchaudio

class Transforms():
    #def __init__(self, sample_rate=44100, n_fft=2048, hop_length=1024, win_length=2048, window='hann', center=True, pad_mode='reflect', power=2.0, n_mels=128, fmin=0.0, fmax=None, htk=False, norm=1, top_db=80.0, ref=1.0, amin=1e-10):
    def __init__(self, sample_rate=44100, n_fft=2048, hop_length=1024, win_length=2048, window='hann', center=True, pad_mode='reflect', power=2.0, n_mels=32, fmin=0.0, fmax=None, htk=False, norm=1, top_db=80.0, ref=1.0, amin=1e-10):
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.win_length = win_length
        self.window = window
        self.center = center
        self.pad_mode = pad_mode
        self.power = power
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        self.htk = htk
        self.norm = norm
        self.top_db = top_db
        self.ref = ref
        self.amin = amin
        #self.mfcc_transform = torchaudio.transforms.MelSpectrogram(sample_rate = 44100, n_fft = 2048, win_length= 2048, hop_length=1024, n_mels=64)

    def __call__(self, x): 
        #x = torch.from_numpy(x)
        #x = torch.Tensor(x)
        x = torch.tensor(x)
        x = x.permute(1, 0)
        x = self.stft(x)
        x = self.mel_spectrogram(x)
        x = self.amplitude_to_db(x)
        return x

    def stft(self, x, n_fft=2048, hop_length=1024, win_length=2048, window=torch.hann_window(2048, device = torch.device('cuda')), center=True, pad_mode='reflect'):
        return torch.stft(x, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, center=center, pad_mode=pad_mode, onesided=True, return_complex=False)

    def istft(self, x, n_fft=2048, hop_length=1024, win_length=2048, window=torch.hann_window(2048,device = torch.device('cuda')), center=True):
        return torch.istft(x, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, center=center, onesided=True)

    def mel_spectrogram(self, x, sr=44100, n_fft=2048, hop_length = 1024, win_length = 2048, window = 'hann', n_mels=32):
       # return self.mfcc_transform(x)
        return torch.as_tensor(librosa.feature.melspectrogram(x.detach().cpu().numpy(), sr = 44100, n_fft = n_fft, hop_length = hop_length, window = window, n_mels = n_mels, win_length = win_length))
    def chromagram(self, x, sr = 44100, n_fft = 2048, hop_length = 1024, win_length = 2048, window = 'hann'):
        device = x.get_device()
        return torch.as_tensor(librosa.feature.chroma_stft(x.detach().cpu().numpy(), sr=sr, n_fft = n_fft, hop_length = hop_length, win_length = win_length, window = window), device = device)
    def amplitude_to_db(self, x, top_db=80.0, ref=1.0, amin=1e-10):
        return librosa.amplitude_to_db(x, top_db=top_db, ref=ref, amin=amin)

    def RMS(self, audio_tensor):
        # this function is used to calculate the RMS of the audio signal
        # the input will be an audio tensor of shape (num_samples)
        # Note that this function is intended to use on smaller audio signals, and that
        # typically, longer signals get windows similar to the STFT in order to calculate more localized
        # RMS. The purpose of using this was for silent signal detection.
        squared_tensor = torch.pow(audio_tensor, 2)
        mean_power = torch.mean(squared_tensor)
        rms = torch.sqrt(mean_power)
        return rms


In [17]:
# This class should define the training loop for the model. We will define a training loop function
# as well as a train_one_epoch function. This is based on the tutorial on the official PyTorch website: 
# https://pytorch.org/tutorials/beginner/introyt/trainingyt.html#the-training-loop
# we will also implement our training via train/validate/test splits, and save our best models parameters.
# I will use SummaryWriter, a useful class for reporting our data from PyTorch.
import torch
from torch.utils.tensorboard import SummaryWriter

class Train():
    def __init__(self, train_dataset, validation_dataset, summary_writer, optimizer, model, learning_rate, loss, epochs = 1000, report_loss_frequency=15):
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, collate_fn=collate)
        self.validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, collate_fn=collate)
        self.writer = summary_writer
        self.model = model
        self.optimizer = optimizer
        self.learning_rate = learning_rate
        self.loss = loss
        self.report_loss_frequency = report_loss_frequency
        self.run_model(epochs=epochs)

    def train_one_epoch(self, epoch_number):
        epoch_loss = 0.0
        last_loss = 0.0
        for i, data in enumerate(self.train_dataloader):
            # We will use the PyTorch Data Loader class to easily iterate through the data and collect our batches.
            # See musdb.py for a deeper explanation of how we collect batches. Simply speaking, we take a an arbitrary
            # fixed-length part of the song and we split it into fixed length segments, taking the STFT of each. 
            mixture_stft, real_stems_stft = data
            # PyTorch accumulates gradients by default, so we zero out the gradients before each batch update in order to 
            # ensure that the optimizer only uses the gradients from the current batch during the update step.
            # This also should help with performance and memory costs.
            self.optimizer.zero_grad()
            # We call our model to make a prediction
            predicted_stems_stft = self.model(mixture_stft)
            # We compute the loss from our loss function defined in loss.py
            loss = self.loss(predicted_stems_stft, real_stems_stft)
            # We calculate the gradient
            loss.backward()
            # Then update the weights based on the gradient
            self.optimizer.step()
            # In order to report our loss, we print the loss every 15 batches
            accumulating_loss += loss.item()
            if i % self.report_loss_frequency == self.report_loss_frequency - 1:
                last_loss = accumulating_loss
                print("Batch {}. Average loss over last {} batches is: {}".format(i+1, last_loss))
                writer_index = epoch_number * len(self.train_dataloader) + i + 1
                self.writer.add_scalar('Loss/train', last_loss, writer_index)
                accumulating_loss = 0
            
            return last_loss

    def run_model(self, epochs):
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        epoch_number = 0
        #set arbitrarily large initial validation loss
        best_vloss = 2.0 ** 30
        
        for epoch in range(epochs):
            print("STARTING EPOCH {}".format(epoch_number + 1))
            # We want to learn in our learning loops, so we make sure our gradients are updating during each epoch.
            self.model.train(True)
            avg_loss = self.train_one_epoch(epoch_number)
            # We don't need or want the gradient opertaion on any of the 
            self.model.train(False)

            accumulating_vloss = 0.0

            for i, vdata in enumerate(self.validation_dataloader):
                vmixture_stft, vreal_stems_stft = vdata
                vpredicted_stems_stft = self.model(vmixture_stft)
                vloss = self.loss(vpredicted_stems_stft, vreal_stems_stft)
                accumulating_vloss += vloss
            
            avg_vloss = accumulating_vloss / (i + 1)
            print('LOSS training set: {} validation set: {}'.format(avg_loss, avg_vloss))
            
            # log all of this 
            self.writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
            self.writer.flush()

            if avg_vloss < best_vloss:
                best_vloss = avg_vloss
                model_path = 'model_{}_{}'.format(timestamp, epoch_number)
                torch.save(self.model.state_dict(), model_path)
            
            epoch_number += 1

In [18]:
# I will use the museval package to calculate the SDR, SIR, and SAR scores as well as
# the time invariant SDR, SIR SAR variants.
# Importantly, this class will allow us to test the model on full songs. For training, we make some sacrifices to opitimize 
# computations and introduce randomization and we end up processing the song in parts over training. This is also because our reconstruction
# task is to recreate the segments of the song that were tested and to compare only with those features.
import torch
import museval 

def load_model(model_path):
    model = my_model()
    model.load_state_dict(model_path)
    model.eval()
    return model

class Evaluate():
    def __init__(self, test_dataset, model_path):
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, collate_fn = collate)
        self.model = load_model(model_path)
        self.mus = test_dataset.mus

    def reconstruct_all_and_evaluate(self):
        with torch.no_grad():
            for i, data in enumerate(self.test_dataloader):
                mixture_stft, _ = data
                predicted_stems_stft = self.model(mixture_stft)
                self.reconstruct_one_and_evaluate(predicted_stems_stft, i)

    def reconstruct_one_and_evaluate(self, predicted_stems_stfts, i): 
        stem_estimates_time_signals = self.reconstruct_one_song(predicted_stems_stfts)
        self.evaluate_one_song(stem_estimates_time_signals, i)

    def evaluate_one_song(self, stems_estimates_time_signals, i):
        # this function will evaluate the song using the museval package.
        # it will receive the time signals of the stems and the song number in the dataset.
        # it will both save the results in the results and print the scores after evaluating each song.
        scores = museval.eval_mus_track(self.mus[i], stems_estimates_time_signals, output_dir='results')   
        print(scores)

    def reconstruct_one_song(self, predicted_stems_stft):
        # this function will reconstruct the song from the predicted stems stft and return the time signals of the stems.
        # We will use the overlap and add method to reconstruct the song after we recover the time signal for each segment for each stem.
        time_signals = batch_time_signals(predicted_stems_stft)
        stem_estimates_time_signals = self.overlap_and_add(time_signals)
        return stem_estimates_time_signals

    def overlap_and_add(self, time_signals):
        # receives all the reconstructed time signals and overlaps them into one signal to reconstruct the original song.
        # this will happen based on the parameters in the dataset for segment length and overlap.
        step_in_samples = time_signals.shape[2] * (1 - self.test_dataset.overlap)
        segment_length = time_signals.shape[2]
        num_segments = time_signals.shape[1]
        full_signal_length = segment_length + step_in_samples * (num_segments - 1)
        new_signal = torch.zeros((time_signals.shape[0], full_signal_length))
        for i in range(num_segments):
            new_signal[:, i * step_in_samples : i * step_in_samples + segment_length] += time_signals[:, i, :]
        return new_signal




In [ ]:
zz = torch.Tensor(10043)
zz.numpy()

In [ ]:
int(torch.as_tensor(0.43))